In [1]:
import math
import numpy as np
import gurobipy as gb
import pandas as pd
from gurobipy import GRB
import networkx as nx
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv('C:/Users/Huawei/Desktop/study-1/io/n15-k4-Q20-1.csv')


In [3]:
def subtourelim(model, where):
    n = 15
    cars = 4
    if where == GRB.Callback.MIPSOL:
        for car in range(cars):
#             print("SLomalos'")
                # make a list of edges selected in the solution
            vals = model.cbGetSolution(model._vars)

            selected = gb.tuplelist((i, j) for i in range(n + 1) for j in range(n + 1) if vals[(i, j, car)] > 0.5)

            # find the shortest cycle in the selected edge list
            tour = subtour(selected)
            if len(tour) < n + 1:
                # add subtour elimination constr. for every pair of cities in subtour
                model.cbLazy(gb.quicksum(model._vars[(i, j, car)] for i in range(n + 1) for j in range(n + 1)) <= len(tour) - 1)
# Given a tuplelist of edges, find the shortest subtour

def subtour(edges):
    n = 15
    unvisited = [i for i in range(n + 1)]
#     unvisited = capitals[:]
#     cycle = capitals[:] 
    cycle = [i for i in range(n + 1)]
    
    # Dummy - guaranteed to be replaced
    while unvisited:  # true if list is non-empty
        thiscycle = []
        neighbors = unvisited
        while neighbors:
            current = neighbors[0]
            thiscycle.append(current)
            unvisited.remove(current)
            neighbors = [j for i, j in edges.select(current, '*') if j in unvisited]
        if len(thiscycle) >= len(cycle):
            cycle = thiscycle # New shortest subtour
    return cycle

In [ ]:
def calculate_len(x, y):
    return np.rint(math.sqrt((x[0] - y[0]) ** 2 + (x[1] - y[1]) ** 2))

def get_len(df):
    len_ = np.zeros((n , n))
    for i in range(n):
        for j in range(n):
            len_[i,j] = calculate_len([df['x'][i], df['y'][i]], [df['x'][j], df['y'][j]])
    return len_


def komi_model(df, n, k, Q):
#     парсинг названия
#     mass = name.split('_')
#     n = int(mass[0][1:]) + 1
#     k = int(mass[1][1:])
#     Q = int(mass[2][1:])
#     
#     q = df["q_i"].to_list()
#     p = df["p_i"].to_list()
#     c = np.ones(k)
    Q_ = np.ones(k) * Q
    q = df["demand"].to_list()
    p = df["income"].to_list()
    c = np.ones(k)
    len_ = get_len(df)
    
    model = gb.Model("model")

    x = model.addMVar(shape=(n, n, k), vtype=GRB.BINARY)
    carVisit = model.addMVar(shape=(n, k), vtype=GRB.BINARY)
    iService= model.addMVar(shape=(n), vtype=gb.GRB.BINARY)
    qReal = model.addMVar(shape=(n, k), lb=0, vtype=gb.GRB.INTEGER)
    loop = model.addMVar(shape=(k, n, n), lb=0, vtype=gb.GRB.INTEGER)
    
   
    for k_ in range(k):
        model.addConstr(sum(x[0,:,k_]) == 1)
        model.addConstr(sum(x[:,0,k_]) == 1)
        
    for k_ in range (k):
        for i in range (n):
            model.addConstr(sum(x[i,:,k_]) == sum(x[:,i,k_]))
            model.addConstr(x[i,i,k_] == 0)

            model.addConstr(sum(x[i,:,k_]) >= 0)
            model.addConstr(sum(x[i,:,k_]) <= 1)
            model.addConstr(sum(x[:,i,k_]) >= 0)
            model.addConstr(sum(x[:,i,k_]) <= 1)
            
            model.addConstr(qReal[i,k_] <= q[i] * carVisit[i,k_])
            
    for k_ in range (k):
        model.addConstr(sum(qReal[:,k_]) <= Q)
    for i in range (n):
        model.addConstr(sum(qReal[i,:]) <= q[i])
        
    model.addConstr(sum(iService) <= 0.5 * n)
    
        
    for k_ in range(k):
        for i in range(n):
            for j in range(n):
                model.addConstr(loop[k_,i,j] - x[i, j,k_] * (n - 1) <= 0)        
        model.addConstr(sum(loop[k_,0,:]) == sum(carVisit[:,k_]) - 1)
        
    for k_ in range(k):
        for i in range(1, n):
            sum_one = 0
            sum_two = 0
            for j in range(n):
                sum_one += loop[k_,i,j]
                sum_two += loop[k_,j,i]
            model.addConstr(sum_one - sum_two == -carVisit[i,k_])
    
    TC = 0
    for k_ in range(k):
        a = 0
        for i in range(n):
            for j in range(n):
                a += x[i,j,k_] * c[k_] * len_[i,j]
        TC += a

    TR = 0
    for k_ in range(k):
        a = 0
        for i in range(1, n):
            a += qReal[i,k_] / q[i] * p[i]
        TR += a
            
    for k_ in range (k):
        for i in range(1, n ):
            for j in range(1, n):
                if i != j:
                    model.addConstr(qReal[j,k_] * p[j]
                                    >= len_[i,j] * q[j] * x[i,j,k_])

    prior = np.array(p)[1:] / np.array(q)[1:]
    prior_ = dict([(idx,val) for idx,val in enumerate(prior)])
    prior_ = dict(sorted(prior_.items(), key=lambda item: item[1],reverse=True))
    print(prior_)
    for i in range(1,len(list(prior_.keys()))):
            model.addConstr(sum([qReal[list(prior_.keys())[i-1], k_] * carVisit[list(prior_.keys())[i-1],k_ ] for k_ in range(k)])  >= sum([qReal[list(prior_.keys())[i], k_] * carVisit[list(prior_.keys())[i], k_] for k_ in range(k)]))
                
    model.setObjective(TR - TC, gb.GRB.MAXIMIZE)
    model.update()
#     model.tune()
    model.setParam('Heuristics', 0.001)
    model.setParam('GomoryPasses', 0)
   
#     model._vars = x
#     model.Params.lazyConstraints = 1
#     model.optimize(subtourelim)
    model.optimize()
    return model, x, carVisit

### без параметров

In [6]:
df = pd.read_csv('C:/Users/Huawei/Desktop/study-1/io/n15-k4-Q20-1.csv')

n = 16
k = 4

Q = 20

     

komi_model(df, n, k, Q)

Set parameter Username
Academic license - for non-commercial use only - expires 2023-11-15
{12: 5.5, 14: 5.181818181818182, 13: 4.684210526315789, 2: 4.625, 11: 4.571428571428571, 7: 4.5, 8: 4.5, 10: 4.285714285714286, 9: 3.875, 6: 3.8666666666666667, 4: 3.8181818181818183, 1: 3.6666666666666665, 3: 3.652173913043478, 0: 3.0526315789473686, 5: 2.806451612903226}
Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (win64)

CPU model: 12th Gen Intel(R) Core(TM) i7-12700H, instruction set [SSE2|AVX|AVX2]
Thread count: 14 physical cores, 20 logical processors, using up to 20 threads

Optimize a model with 2405 rows, 2192 columns and 12192 nonzeros
Model fingerprint: 0x45a1e6db
Model has 14 quadratic constraints
Variable types: 0 continuous, 2192 integer (1104 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+03]
  QMatrix range    [1e+00, 1e+00]
  Objective range  [3e+00, 5e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+01]
Presolve removed 1106 rows and 252 

[<gurobi.Model MIP instance model: 2405 constrs, 2192 vars, Parameter changes: Username=(user-defined)>,
 <MVar (16, 16, 4)>
 array([[[<gurobi.Var C0 (value -0.0)>, <gurobi.Var C1 (value -0.0)>,
          <gurobi.Var C2 (value -0.0)>, <gurobi.Var C3 (value -0.0)>],
         [<gurobi.Var C4 (value 1.0)>, <gurobi.Var C5 (value 0.0)>,
          <gurobi.Var C6 (value -0.0)>, <gurobi.Var C7 (value -0.0)>],
         [<gurobi.Var C8 (value 0.0)>, <gurobi.Var C9 (value -0.0)>,
          <gurobi.Var C10 (value -0.0)>, <gurobi.Var C11 (value 1.0)>],
         ...,
         [<gurobi.Var C52 (value 0.0)>, <gurobi.Var C53 (value -0.0)>,
          <gurobi.Var C54 (value -0.0)>, <gurobi.Var C55 (value 0.0)>],
         [<gurobi.Var C56 (value 0.0)>, <gurobi.Var C57 (value -0.0)>,
          <gurobi.Var C58 (value -0.0)>, <gurobi.Var C59 (value -0.0)>],
         [<gurobi.Var C60 (value -0.0)>, <gurobi.Var C61 (value -0.0)>,
          <gurobi.Var C62 (value -0.0)>, <gurobi.Var C63 (value -0.0)>]],
 
     

In [7]:
komi_model(df, n, k, Q)

{12: 5.5, 14: 5.181818181818182, 13: 4.684210526315789, 2: 4.625, 11: 4.571428571428571, 7: 4.5, 8: 4.5, 10: 4.285714285714286, 9: 3.875, 6: 3.8666666666666667, 4: 3.8181818181818183, 1: 3.6666666666666665, 3: 3.652173913043478, 0: 3.0526315789473686, 5: 2.806451612903226}
Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (win64)

CPU model: 12th Gen Intel(R) Core(TM) i7-12700H, instruction set [SSE2|AVX|AVX2]
Thread count: 14 physical cores, 20 logical processors, using up to 20 threads

Optimize a model with 2405 rows, 2192 columns and 12192 nonzeros
Model fingerprint: 0x45a1e6db
Model has 14 quadratic constraints
Variable types: 0 continuous, 2192 integer (1104 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+03]
  QMatrix range    [1e+00, 1e+00]
  Objective range  [3e+00, 5e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+01]
Presolve removed 1106 rows and 252 columns
Presolve time: 0.03s
Presolved: 1460 rows, 1996 columns, 8744 nonzeros
Variable typ

[<gurobi.Model MIP instance model: 2405 constrs, 2192 vars, Parameter changes: Username=(user-defined)>,
 <MVar (16, 16, 4)>
 array([[[<gurobi.Var C0 (value -0.0)>, <gurobi.Var C1 (value -0.0)>,
          <gurobi.Var C2 (value -0.0)>, <gurobi.Var C3 (value -0.0)>],
         [<gurobi.Var C4 (value 1.0)>, <gurobi.Var C5 (value 0.0)>,
          <gurobi.Var C6 (value -0.0)>, <gurobi.Var C7 (value -0.0)>],
         [<gurobi.Var C8 (value 0.0)>, <gurobi.Var C9 (value -0.0)>,
          <gurobi.Var C10 (value -0.0)>, <gurobi.Var C11 (value 1.0)>],
         ...,
         [<gurobi.Var C52 (value 0.0)>, <gurobi.Var C53 (value -0.0)>,
          <gurobi.Var C54 (value -0.0)>, <gurobi.Var C55 (value 0.0)>],
         [<gurobi.Var C56 (value 0.0)>, <gurobi.Var C57 (value -0.0)>,
          <gurobi.Var C58 (value -0.0)>, <gurobi.Var C59 (value -0.0)>],
         [<gurobi.Var C60 (value -0.0)>, <gurobi.Var C61 (value -0.0)>,
          <gurobi.Var C62 (value -0.0)>, <gurobi.Var C63 (value -0.0)>]],
 
     

In [8]:
komi_model(df, n, k, Q)

{12: 5.5, 14: 5.181818181818182, 13: 4.684210526315789, 2: 4.625, 11: 4.571428571428571, 7: 4.5, 8: 4.5, 10: 4.285714285714286, 9: 3.875, 6: 3.8666666666666667, 4: 3.8181818181818183, 1: 3.6666666666666665, 3: 3.652173913043478, 0: 3.0526315789473686, 5: 2.806451612903226}
Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (win64)

CPU model: 12th Gen Intel(R) Core(TM) i7-12700H, instruction set [SSE2|AVX|AVX2]
Thread count: 14 physical cores, 20 logical processors, using up to 20 threads

Optimize a model with 2405 rows, 2192 columns and 12192 nonzeros
Model fingerprint: 0x45a1e6db
Model has 14 quadratic constraints
Variable types: 0 continuous, 2192 integer (1104 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+03]
  QMatrix range    [1e+00, 1e+00]
  Objective range  [3e+00, 5e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+01]
Presolve removed 1106 rows and 252 columns
Presolve time: 0.02s
Presolved: 1460 rows, 1996 columns, 8744 nonzeros
Variable typ

[<gurobi.Model MIP instance model: 2405 constrs, 2192 vars, Parameter changes: Username=(user-defined)>,
 <MVar (16, 16, 4)>
 array([[[<gurobi.Var C0 (value -0.0)>, <gurobi.Var C1 (value -0.0)>,
          <gurobi.Var C2 (value -0.0)>, <gurobi.Var C3 (value -0.0)>],
         [<gurobi.Var C4 (value 1.0)>, <gurobi.Var C5 (value 0.0)>,
          <gurobi.Var C6 (value -0.0)>, <gurobi.Var C7 (value -0.0)>],
         [<gurobi.Var C8 (value 0.0)>, <gurobi.Var C9 (value -0.0)>,
          <gurobi.Var C10 (value -0.0)>, <gurobi.Var C11 (value 1.0)>],
         ...,
         [<gurobi.Var C52 (value 0.0)>, <gurobi.Var C53 (value -0.0)>,
          <gurobi.Var C54 (value -0.0)>, <gurobi.Var C55 (value 0.0)>],
         [<gurobi.Var C56 (value 0.0)>, <gurobi.Var C57 (value -0.0)>,
          <gurobi.Var C58 (value -0.0)>, <gurobi.Var C59 (value -0.0)>],
         [<gurobi.Var C60 (value -0.0)>, <gurobi.Var C61 (value -0.0)>,
          <gurobi.Var C62 (value -0.0)>, <gurobi.Var C63 (value -0.0)>]],
 
     

### с параметрами:

In [10]:
df = pd.read_csv('C:/Users/Huawei/Desktop/study-1/io/n15-k4-Q20-1.csv')

n = 16
k = 4

Q = 20

     

komi_model(df, n, k, Q)

{12: 5.5, 14: 5.181818181818182, 13: 4.684210526315789, 2: 4.625, 11: 4.571428571428571, 7: 4.5, 8: 4.5, 10: 4.285714285714286, 9: 3.875, 6: 3.8666666666666667, 4: 3.8181818181818183, 1: 3.6666666666666665, 3: 3.652173913043478, 0: 3.0526315789473686, 5: 2.806451612903226}
Set parameter Heuristics to value 0.001
Set parameter GomoryPasses to value 0
Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (win64)

CPU model: 12th Gen Intel(R) Core(TM) i7-12700H, instruction set [SSE2|AVX|AVX2]
Thread count: 14 physical cores, 20 logical processors, using up to 20 threads

Optimize a model with 2405 rows, 2192 columns and 12192 nonzeros
Model fingerprint: 0x45a1e6db
Model has 14 quadratic constraints
Variable types: 0 continuous, 2192 integer (1104 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+03]
  QMatrix range    [1e+00, 1e+00]
  Objective range  [3e+00, 5e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+01]
Presolve removed 1106 rows and 252 columns
Preso

(<gurobi.Model MIP instance model: 2405 constrs, 2192 vars, Parameter changes: Heuristics=0.001, GomoryPasses=0, Username=(user-defined)>,
 <MVar (16, 16, 4)>
 array([[[<gurobi.Var C0 (value 0.0)>, <gurobi.Var C1 (value 0.0)>,
          <gurobi.Var C2 (value 0.0)>, <gurobi.Var C3 (value 0.0)>],
         [<gurobi.Var C4 (value 0.0)>, <gurobi.Var C5 (value 1.0)>,
          <gurobi.Var C6 (value 0.0)>, <gurobi.Var C7 (value 0.0)>],
         [<gurobi.Var C8 (value 0.0)>, <gurobi.Var C9 (value 0.0)>,
          <gurobi.Var C10 (value 0.0)>, <gurobi.Var C11 (value 1.0)>],
         ...,
         [<gurobi.Var C52 (value 0.0)>, <gurobi.Var C53 (value 0.0)>,
          <gurobi.Var C54 (value 0.0)>, <gurobi.Var C55 (value 0.0)>],
         [<gurobi.Var C56 (value 0.0)>, <gurobi.Var C57 (value 0.0)>,
          <gurobi.Var C58 (value 0.0)>, <gurobi.Var C59 (value 0.0)>],
         [<gurobi.Var C60 (value 0.0)>, <gurobi.Var C61 (value 0.0)>,
          <gurobi.Var C62 (value 0.0)>, <gurobi.Var C63 (value

In [11]:
n = 16
k = 4

Q = 20

     

komi_model(df, n, k, Q)

{12: 5.5, 14: 5.181818181818182, 13: 4.684210526315789, 2: 4.625, 11: 4.571428571428571, 7: 4.5, 8: 4.5, 10: 4.285714285714286, 9: 3.875, 6: 3.8666666666666667, 4: 3.8181818181818183, 1: 3.6666666666666665, 3: 3.652173913043478, 0: 3.0526315789473686, 5: 2.806451612903226}
Set parameter Heuristics to value 0.001
Set parameter GomoryPasses to value 0
Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (win64)

CPU model: 12th Gen Intel(R) Core(TM) i7-12700H, instruction set [SSE2|AVX|AVX2]
Thread count: 14 physical cores, 20 logical processors, using up to 20 threads

Optimize a model with 2405 rows, 2192 columns and 12192 nonzeros
Model fingerprint: 0x45a1e6db
Model has 14 quadratic constraints
Variable types: 0 continuous, 2192 integer (1104 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+03]
  QMatrix range    [1e+00, 1e+00]
  Objective range  [3e+00, 5e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+01]
Presolve removed 1106 rows and 252 columns
Preso

(<gurobi.Model MIP instance model: 2405 constrs, 2192 vars, Parameter changes: Heuristics=0.001, GomoryPasses=0, Username=(user-defined)>,
 <MVar (16, 16, 4)>
 array([[[<gurobi.Var C0 (value 0.0)>, <gurobi.Var C1 (value 0.0)>,
          <gurobi.Var C2 (value 0.0)>, <gurobi.Var C3 (value 0.0)>],
         [<gurobi.Var C4 (value 0.0)>, <gurobi.Var C5 (value 1.0)>,
          <gurobi.Var C6 (value 0.0)>, <gurobi.Var C7 (value 0.0)>],
         [<gurobi.Var C8 (value 0.0)>, <gurobi.Var C9 (value 0.0)>,
          <gurobi.Var C10 (value 0.0)>, <gurobi.Var C11 (value 1.0)>],
         ...,
         [<gurobi.Var C52 (value 0.0)>, <gurobi.Var C53 (value 0.0)>,
          <gurobi.Var C54 (value 0.0)>, <gurobi.Var C55 (value 0.0)>],
         [<gurobi.Var C56 (value 0.0)>, <gurobi.Var C57 (value 0.0)>,
          <gurobi.Var C58 (value 0.0)>, <gurobi.Var C59 (value 0.0)>],
         [<gurobi.Var C60 (value 0.0)>, <gurobi.Var C61 (value 0.0)>,
          <gurobi.Var C62 (value 0.0)>, <gurobi.Var C63 (value

In [12]:
n = 16
k = 4

Q = 20

     

komi_model(df, n, k, Q)

{12: 5.5, 14: 5.181818181818182, 13: 4.684210526315789, 2: 4.625, 11: 4.571428571428571, 7: 4.5, 8: 4.5, 10: 4.285714285714286, 9: 3.875, 6: 3.8666666666666667, 4: 3.8181818181818183, 1: 3.6666666666666665, 3: 3.652173913043478, 0: 3.0526315789473686, 5: 2.806451612903226}
Set parameter Heuristics to value 0.001
Set parameter GomoryPasses to value 0
Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (win64)

CPU model: 12th Gen Intel(R) Core(TM) i7-12700H, instruction set [SSE2|AVX|AVX2]
Thread count: 14 physical cores, 20 logical processors, using up to 20 threads

Optimize a model with 2405 rows, 2192 columns and 12192 nonzeros
Model fingerprint: 0x45a1e6db
Model has 14 quadratic constraints
Variable types: 0 continuous, 2192 integer (1104 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+03]
  QMatrix range    [1e+00, 1e+00]
  Objective range  [3e+00, 5e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+01]
Presolve removed 1106 rows and 252 columns
Preso

(<gurobi.Model MIP instance model: 2405 constrs, 2192 vars, Parameter changes: Heuristics=0.001, GomoryPasses=0, Username=(user-defined)>,
 <MVar (16, 16, 4)>
 array([[[<gurobi.Var C0 (value 0.0)>, <gurobi.Var C1 (value 0.0)>,
          <gurobi.Var C2 (value 0.0)>, <gurobi.Var C3 (value 0.0)>],
         [<gurobi.Var C4 (value 0.0)>, <gurobi.Var C5 (value 1.0)>,
          <gurobi.Var C6 (value 0.0)>, <gurobi.Var C7 (value 0.0)>],
         [<gurobi.Var C8 (value 0.0)>, <gurobi.Var C9 (value 0.0)>,
          <gurobi.Var C10 (value 0.0)>, <gurobi.Var C11 (value 1.0)>],
         ...,
         [<gurobi.Var C52 (value 0.0)>, <gurobi.Var C53 (value 0.0)>,
          <gurobi.Var C54 (value 0.0)>, <gurobi.Var C55 (value 0.0)>],
         [<gurobi.Var C56 (value 0.0)>, <gurobi.Var C57 (value 0.0)>,
          <gurobi.Var C58 (value 0.0)>, <gurobi.Var C59 (value 0.0)>],
         [<gurobi.Var C60 (value 0.0)>, <gurobi.Var C61 (value 0.0)>,
          <gurobi.Var C62 (value 0.0)>, <gurobi.Var C63 (value

с колбэк и параметрами


In [ ]:
n = 16
k = 4

Q = 20

     

komi_model(df, n, k, Q)

In [ ]:

komi_model(df, n, k, Q)

In [14]:

komi_model(df, n, k, Q)

{12: 5.5, 14: 5.181818181818182, 13: 4.684210526315789, 2: 4.625, 11: 4.571428571428571, 7: 4.5, 8: 4.5, 10: 4.285714285714286, 9: 3.875, 6: 3.8666666666666667, 4: 3.8181818181818183, 1: 3.6666666666666665, 3: 3.652173913043478, 0: 3.0526315789473686, 5: 2.806451612903226}
Set parameter Heuristics to value 0.001
Set parameter GomoryPasses to value 0
Set parameter LazyConstraints to value 1
Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (win64)

CPU model: 12th Gen Intel(R) Core(TM) i7-12700H, instruction set [SSE2|AVX|AVX2]
Thread count: 14 physical cores, 20 logical processors, using up to 20 threads

Optimize a model with 2405 rows, 2192 columns and 12192 nonzeros
Model fingerprint: 0x45a1e6db
Model has 14 quadratic constraints
Variable types: 0 continuous, 2192 integer (1104 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+03]
  QMatrix range    [1e+00, 1e+00]
  Objective range  [3e+00, 5e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+01]
Presolv

(<gurobi.Model MIP instance model: 2405 constrs, 2192 vars, Parameter changes: Heuristics=0.001, GomoryPasses=0, Username=(user-defined), LazyConstraints=1>,
 <MVar (16, 16, 4)>
 array([[[<gurobi.Var C0 (value -0.0)>, <gurobi.Var C1 (value -0.0)>,
          <gurobi.Var C2 (value -0.0)>, <gurobi.Var C3 (value -0.0)>],
         [<gurobi.Var C4 (value -0.0)>, <gurobi.Var C5 (value 1.0)>,
          <gurobi.Var C6 (value 1.0)>, <gurobi.Var C7 (value 0.0)>],
         [<gurobi.Var C8 (value -0.0)>, <gurobi.Var C9 (value 0.0)>,
          <gurobi.Var C10 (value -0.0)>,
          <gurobi.Var C11 (value 0.9999999999999998)>],
         ...,
         [<gurobi.Var C52 (value -0.0)>, <gurobi.Var C53 (value -0.0)>,
          <gurobi.Var C54 (value -0.0)>, <gurobi.Var C55 (value -0.0)>],
         [<gurobi.Var C56 (value 0.0)>, <gurobi.Var C57 (value -0.0)>,
          <gurobi.Var C58 (value -0.0)>, <gurobi.Var C59 (value -0.0)>],
         [<gurobi.Var C60 (value -0.0)>, <gurobi.Var C61 (value -0.0)>,
  

другая без параметров

In [ ]:
df = pd.read_csv('C:/Users/Huawei/Desktop/study-1/io/n39-k6-Q40-1.csv') 
n = 40
k = 6

Q = 40

     

komi_model(df, n, k, Q)

{35: 4.5, 16: 4.333333333333333, 20: 4.25, 38: 4.0, 25: 3.857142857142857, 30: 3.6363636363636362, 34: 3.588235294117647, 32: 3.391304347826087, 18: 3.3333333333333335, 14: 3.3, 19: 3.2857142857142856, 9: 3.2, 33: 3.1538461538461537, 36: 3.111111111111111, 6: 3.1052631578947367, 27: 3.0714285714285716, 2: 3.0, 12: 3.0, 28: 3.0, 29: 3.0, 15: 2.933333333333333, 22: 2.875, 23: 2.7, 0: 2.5714285714285716, 7: 2.5652173913043477, 21: 2.5, 24: 2.392857142857143, 13: 2.380952380952381, 8: 2.3636363636363638, 1: 2.2666666666666666, 5: 2.2, 37: 2.1333333333333333, 3: 2.111111111111111, 4: 2.0476190476190474, 17: 1.9268292682926829, 10: 1.7894736842105263, 31: 1.75, 11: 1.7241379310344827, 26: 1.6666666666666667}
Set parameter Heuristics to value 0.001
Set parameter GomoryPasses to value 0
Set parameter LazyConstraints to value 1
Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (win64)

CPU model: 12th Gen Intel(R) Core(TM) i7-12700H, instruction set [SSE2|AVX|AVX2]
Thread count: 14 physical core

KeyboardInterrupt: 

Exception ignored in: 'gurobipy.callbackstub'
Traceback (most recent call last):
  File "src\gurobipy\callback.pxi", line 209, in gurobipy.CallbackClass.callback
  File "C:\Users\Huawei\AppData\Local\Temp\ipykernel_31524\1796921481.py", line 1, in subtourelim
KeyboardInterrupt: 


 14654 10884  304.09669   57  429  169.70744  334.27464  97.0%   301  143s
 15169 11756  290.70014   93  455  169.70744  334.27464  97.0%   299  147s
 16087 12616  272.77426  175  363  169.70744  334.27464  97.0%   294  152s
 17083 13377  218.30832  262  366  169.70744  333.09576  96.3%   289  156s
 17935 13892  268.95100   62  323  169.70744  333.09576  96.3%   287  161s
 18516 14841  207.33672  124  247  169.70744  332.61309  96.0%   288  165s
 19594 14849  197.55211  161  585  169.70744  332.51438  95.9%   284  170s
 19603 14855  285.95014   82  595  169.70744  332.51438  95.9%   284  175s
 19721 14949  332.51438   31  452  169.70744  332.51438  95.9%   289  180s
H19789 14243                     170.7074433  332.51438  94.8%   289  181s
H19806 13532                     171.1557191  332.51438  94.3%   289  181s
 19996 13657  332.50186   38  450  171.15572  332.51438  94.3%   289  185s
 20310 13835  332.51438   46  540  171.15572  332.51438  94.3%   289  190s
H20581 13308             

KeyboardInterrupt: 

...

 33904 18665  240.36260   65  197  176.11894  323.67490  83.8%   261  301s
 34786 18865  291.43546   68  412  176.11894  323.20721  83.5%   260  306s
 35339 19351  250.86101  113  431  176.11894  323.20721  83.5%   259  311s
 36127 20245  204.61230  195  309  176.11894  322.59585  83.2%   257  316s
 37437 20301  260.29649   68  502  176.11894  321.76858  82.7%   254  320s
 38122 20849  310.79447   45  598  176.11894  321.76858  82.7%   254  325s
 39097 21203  275.51592   74  427  176.11894  321.17272  82.4%   253  331s
 39953 22303  274.21894   55  400  176.11894  321.10713  82.3%   253  337s
 41498 22709  184.61362  223  275  176.11894  320.50583  82.0%   249  341s
 42517 23005  209.79040   54  292  176.11894  320.42639  81.9%   247  345s
 43275 23310  276.93080   70  352  176.11894  320.42639  81.9%   247  350s
 43948 23952  224.31166  136  300  176.11894  320.10708  81.8%   246  356s
 45011 24311  261.81921   91  358  176.11894  319.67919  81.5%   245  361s
 45817 24932  304.52467  

 122689 56047  250.76533   79  295  183.82818  308.18827  67.7%   289 6169s
 124155 56947  261.32418   89  458  183.82818  308.03906  67.6%   290 6182s
 125778 57361  238.09472   78  184  183.82818  307.87134  67.5%   291 6197s
 127115 57912  207.17328   56  290  183.82818  307.83853  67.5%   292 6215s
 128524 58488  227.64510  113  416  183.82818  307.69376  67.4%   293 6230s
 129812 59225  287.69111   54  554  183.82818  307.58070  67.3%   294 6244s
 131195 59917  285.40433   53  606  183.82818  307.49323  67.3%   295 6257s
 132532 61111  292.44563   52  656  183.82818  307.35749  67.2%   295 6272s
 134361 62153  266.86755   57  363  183.82818  307.25314  67.1%   296 6287s
 135725 63450  266.21573   58  463  183.82818  307.11314  67.1%   296 6304s
 137296 64838  275.86461   63  449  183.82818  307.00216  67.0%   297 6322s
 138971 65889  242.76112   77  425  183.82818  306.84386  66.9%   297 6337s
 140241 67117  254.93647   86  368  183.82818  306.70328  66.8%   298 6357s
 141789 6872

 272472 172180  254.45607   80  248  183.82818  299.33248  62.8%   351 9811s
 273905 173539  252.66711   69  458  183.82818  299.27715  62.8%   352 9874s
 275541 174458  275.31621   47  512  183.82818  299.21436  62.8%   352 9914s
 276682 175621  257.96271   78  394  183.82818  299.15685  62.7%   353 9971s
 278233 177153  235.54803   92  318  183.82818  299.09783  62.7%   353 10037s
 280055 178413  281.84599   59  609  183.82818  299.02735  62.7%   353 10093s
 281692 179680  283.74339   58  624  183.82818  298.95273  62.6%   353 10148s
 283251 180920  267.32156   58  537  183.82818  298.89641  62.6%   353 10209s
 283658 180920  202.44021   88  384  183.82818  298.87559  62.6%   354 10210s
 284809 182024  232.63742   60  375  183.82818  298.83396  62.6%   354 10268s
 286278 182024  255.92647   55  463  183.82818  298.78688  62.5%   354 10270s
 286398 182804  264.91384   67  392  183.82818  298.78246  62.5%   354 10309s
 287477 183753  209.31806   92  307  183.82818  298.70867  62.5%   3

 420927 289339  237.05440   86  305  183.82818  294.34214  60.1%   379 15248s
 422299 290740  243.42254   72  360  183.82818  294.31340  60.1%   379 15275s
 423934 291785  207.12937  117  368  183.82818  294.28074  60.1%   379 15301s
 425303 292987  274.43188   68  549  183.82818  294.22243  60.1%   379 15329s
 426769 294103  216.57146   64  480  183.82818  294.18724  60.0%   379 15359s
 428186 295424  282.12115   64  505  183.82818  294.15291  60.0%   380 15386s
 429740 296750  215.95339  110  474  183.82818  294.11388  60.0%   380 15413s
 431389 297666  252.28361   55  504  183.82818  294.07641  60.0%   380 15437s
 432670 298705  208.67811   77  288  183.82818  294.03879  60.0%   380 15462s
 434030 299857  249.05625   77  331  183.82818  294.01444  59.9%   380 15489s
 435444 300821  270.22162   56  506  183.82818  293.97995  59.9%   381 15514s
 435596 300821  269.38255   75  527  183.82818  293.97544  59.9%   381 15515s
 436704 301671  241.09553   60  474  183.82818  293.94108  59.9%

 575785 411495  245.10032   62  525  183.82818  290.97052  58.3%   396 18157s
 576997 412457  262.87530   72  482  183.82818  290.93336  58.3%   396 18182s
 578413 413601  238.51533   63  442  183.82818  290.89584  58.2%   397 18210s
 579915 414787  268.53380   57  625  183.82818  290.87254  58.2%   397 18239s
 580358 414787  202.05202  105  213  183.82818  290.87028  58.2%   397 18240s
 581478 416137  256.37607   62  425  183.82818  290.84763  58.2%   397 18269s
 583147 417419  243.37504   75  419  183.82818  290.82444  58.2%   397 18298s
 584728 418474  258.51546   72  463  183.82818  290.79205  58.2%   397 18323s
 586110 419842  262.53899   60  498  183.82818  290.76847  58.2%   397 18353s
 587917 420871  253.00099   57  426  183.82818  290.74126  58.2%   397 18378s
 589368 421954  252.29630   66  419  183.82818  290.72627  58.2%   398 18407s
 590962 423127  228.18277   70  229  183.82818  290.70514  58.1%   398 18436s
 592553 424391  267.56169   61  462  183.82818  290.67922  58.1%

 c пареметрами

In [ ]:
komi_model(df, n, k, Q)